### Neural Network - Two Hidden Layers

In [1]:
# Import our dependencies
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from pathlib import Path

In [2]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
def preprocessFile(filename = Path('../Data/AdultCensusUpdated.csv')):
    
    print(f'Preprocessing file {filename}')
    dtype_mapper = {'age': 'int64',
                    'workclass': 'string',
                    'fnlwgt': 'int64',
                    'education': 'string',
                    'education.num': 'int64',
                    'marital.status': 'string',
                    'occupation': 'string',
                    'relationship': 'string',
                    'race': 'string',
                    'sex': 'string',
                    'capital.gain': 'int64',
                    'capital.loss': 'int64',
                    'hours.per.week': 'int64',
                    'native.country': 'string',
                    'income': 'string',
                    'State': 'string'}
    df = pd.read_csv(filename, dtype=dtype_mapper, na_values='?')    
    df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'state']
    df.income = df.income.apply(lambda x: '0' if x == '<=50K' else '1')
    df.income = df.income.astype('int64')
    df = df.drop(['fnlwgt',
                  'education', 
                #   'state',
                  'capital_gain', 
                  'capital_loss',
                  'native_country'], axis=1)
    df = df.dropna()
    return df

census_df = preprocessFile()
census_df.sample(10)

Preprocessing file ..\Data\AdultCensusUpdated.csv


,age,workclass,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,income,state
13333,17,Self-emp-not-inc,6,Never-married,Other-service,Own-child,White,Female,24,0,South Carolina
13473,25,Private,9,Never-married,Adm-clerical,Unmarried,White,Female,40,0,Kansas
20865,41,Private,10,Married-spouse-absent,Sales,Unmarried,Black,Female,40,0,Virginia
11261,21,Private,1,Never-married,Farming-fishing,Not-in-family,White,Male,50,0,Missouri
19512,39,Private,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,1,Michigan
5278,61,Self-emp-inc,14,Married-civ-spouse,Sales,Husband,White,Male,40,1,New Hampshire
28676,27,Private,9,Never-married,Transport-moving,Own-child,White,Male,40,0,Maryland
2926,59,Self-emp-not-inc,10,Widowed,Farming-fishing,Not-in-family,White,Female,40,0,Texas
18464,66,Private,14,Married-civ-spouse,Sales,Husband,White,Male,40,0,Arkansas
19428,54,Self-emp-not-inc,10,Married-civ-spouse,Sales,Husband,White,Male,50,0,Maine


In [4]:
y = census_df.income

X = census_df.copy()
X.drop('income', axis=1, inplace=True)


In [5]:
# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Apply Label Encoding to each non-numeric column
for col in X.select_dtypes(include=["string"]).columns:
    X[col] = label_encoder.fit_transform(X[col])

X.head()

,age,workclass,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,state
1,82,2,9,6,3,1,4,0,18,19
3,54,2,4,0,6,4,4,0,40,11
4,41,2,10,5,9,3,4,0,40,8
5,34,2,9,0,7,4,4,0,45,8
6,38,2,6,5,0,4,4,1,40,4


In [6]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Create scaler instance
X_scaler = scaler.fit(X_train)

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Define the deep learning model 
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                176       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Compile the model
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
720/720 [==============================] - 1s 869us/step - loss: 0.4586 - accuracy: 0.7758
Epoch 2/100
720/720 [==============================] - 1s 899us/step - loss: 0.3782 - accuracy: 0.8209
Epoch 3/100
720/720 [==============================] - 1s 801us/step - loss: 0.3705 - accuracy: 0.8245
Epoch 4/100
720/720 [==============================] - 1s 877us/step - loss: 0.3675 - accuracy: 0.8259
Epoch 5/100
720/720 [==============================] - 1s 835us/step - loss: 0.3654 - accuracy: 0.8266
Epoch 6/100
720/720 [==============================] - 1s 832us/step - loss: 0.3634 - accuracy: 0.8269
Epoch 7/100
720/720 [==============================] - 1s 784us/step - loss: 0.3615 - accuracy: 0.8283
Epoch 8/100
720/720 [==============================] - 1s 782us/step - loss: 0.3604 - accuracy: 0.8274
Epoch 9/100
720/720 [==============================] - 1s 774us/step - loss: 0.3594 - accuracy: 0.8287
Epoch 10/100
720/720 [==============================] - 1s 839us/step - l

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

240/240 - 0s - loss: 0.3741 - accuracy: 0.8249 - 278ms/epoch - 1ms/step
Loss: 0.37412557005882263, Accuracy: 0.8248698115348816
